# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [137]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.proportion import proportions_ztest as ztest


from scipy.stats import ttest_ind, norm, t, ttest_1samp
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [3]:
# Run this code:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [67]:
salaries.head()


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [155]:
# create the sample
# many data is empty, we are going to take only the data where hourly rate is more than 0

salaries_sample = salaries[salaries['Hourly Rate'] > 0]['Hourly Rate'].sample(50)

# H₀: 𝛍₁=𝛍₂   vs   H₁: 𝛍₁≠𝛍₂  
st_number, p_value = ttest_1samp(salaries_sample, 30, alternative='two-sided')
p_value


0.1296684963114742

In [156]:
p_value < 0.05 #False
#H₀ Can't be rejected

False

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [66]:
from math import sqrt
df = salaries[salaries['Hourly Rate'] > 0]
df = df['Hourly Rate']


# Step 1: Calculate the mean of the dataset
mean = np.mean(df)

# Step 2: Calculate the Standard Error (Standard Deviation / sqrt(n))
std = np.std(df)  # Sample standard deviation
n = len(df) # Size of the sample
std_err = std / sqrt(n) # Standar Error

# Step 3: define our confidence level
alpha = 0.95  # Confidence level

# Step 4: Build confidence interval with t.interval()
confidence_interval = t.interval(alpha, df=n-1, loc=mean, scale=std_err)
print(f"Our confidence interval is {confidence_interval} $ per hour")


Our confidence interval is (32.52347486867093, 33.053640563889545) $ per hour


32.78855771628024

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [153]:


# Define H(0) & H(1) H₀: p=0.25   vs   H₁: p≠0.25 
# Define p = proportion of the group and confidence level
p = 0.25
salaries_sample = salaries.sample(3000)
alpha = 0.05

# Calculate the proportion group and the total sample size
hourly_workers = salaries_sample[salaries_sample['Salary or Hourly'] == 'Hourly']['Salary or Hourly'].count()
n = salaries_sample['Salary or Hourly'].count()

# Compute ztest
stats, p_value = ztest(hourly_workers, n, value=p, alternative='two-sided')
p_value

0.7349699715496356

In [154]:
p_value < alpha # False
# H₀: p=0.25 Can't be rejected


False